In [2]:
import pandas as pd
import numpy as np
import uncertainties as unc
from uncertainties import unumpy 
from scipy.signal import medfilt

In [3]:
eps = 1e-12

def _tn(y_true, y_pred):
    return np.logical_and(y_pred == 0, y_true == 0).astype(np.float64).sum()

def _tp(y_true, y_pred):
    return np.logical_and(y_pred == 1, y_true == 1).astype(np.float64).sum()

def _fn(y_true, y_pred):
    return np.logical_and(y_pred == 0, y_true == 1).astype(np.float64).sum()

def _fp(y_true, y_pred):
    return np.logical_and(y_pred == 1, y_true == 0).astype(np.float64).sum()

def fpr(y_true, y_pred):
    fp = _fp(y_true, y_pred)
    tn = _tn(y_true, y_pred)
    return fp / (fp + tn + eps)

def fnr(y_true, y_pred):
    fn = _fn(y_true, y_pred)
    tp = _tp(y_true, y_pred)
    return fn/ (fn + tp + eps)

def tpr(y_true, y_pred):
    return 1 - fnr(y_true, y_pred)

def tnr(y_true, y_pred):
    return 1 - fpr(y_true, y_pred)

# Resultados da Random Forest

In [17]:
rf_results = pd.read_pickle('models/rf-depht12-preds.pkl')

In [25]:
rf_results = pd.read_pickle('models/rf-preds.pkl')

## Grupo controle

Vou primeiramente não fazer nenhum processamento extra e verificar se os valores obtidos batem com resultados já salvos.

### Média e Desvio padrão nos folds

In [26]:
group_sizes = [10, 6, 6, 6, 6, 7, 6, 6, 6]
pos = np.hstack((np.zeros(1), np.cumsum(group_sizes)))

In [27]:
finals = {}
for layer in rf_results.columns.levels[0]:
    finals[layer] = {}
    groups = []
    for idx in range(len(pos)-1):
        layer_data = rf_results.loc[pos[idx]:pos[idx+1]-1, layer]

        metrics_fnr = {}
        metrics_fpr = {}
        metrics_tnr = {}
        metrics_tpr = {}
        
        pred = layer_data.loc[:, 'predictions']
        true = layer_data.loc[:, 'labels']
        
        for idx,( y_true, y_pred) in enumerate(zip(true, pred)):
            metrics_fnr.update({'{}'.format(idx): fnr(y_true, y_pred)})
            metrics_fpr.update({'{}'.format(idx): fpr(y_true, y_pred)})
            metrics_tpr.update({'{}'.format(idx): tpr(y_true, y_pred)})
            metrics_tnr.update({'{}'.format(idx): tnr(y_true, y_pred)})

        groups += [pd.DataFrame({'fnr': metrics_fnr, 'fpr': metrics_fpr, 'tpr': metrics_tpr, 'tnr': metrics_tnr})]

    finals[layer] = pd.concat(groups, keys=['Black_Backpack', 'Black_Coat', 'Brown_Box',
                                            'Camera_Box', 'Dark-Blue_Box', 'Pink_Bottle',
                                            'Shoe', 'Towel', 'White_Jar'])

z = pd.concat(finals)

In [28]:
rf_control_group_stats = z.groupby(level=[0,1]).agg('mean').groupby(level=0).agg(['mean', 'std'])
dis = z.groupby(level=[0,1]).agg('mean').loc[:, ['fnr', 'fpr']].agg(
    lambda x: np.sqrt((x**2).sum()), axis=1).groupby(level=0).agg(['mean', 'std'])

dis.columns = [('dis', name) for name in dis.columns]
rf_control_group_stats = pd.concat([rf_control_group_stats, dis], axis=1)
rf_control_group_stats

fnr                 fpr                 tnr            \
                    mean       std      mean       std      mean       std   
res2a_branch2a  0.320057  0.108386  0.321925  0.138613  0.678075  0.138613   
res2b_branch2a  0.261081  0.125660  0.312946  0.211433  0.687054  0.211433   
res2c_branch2a  0.301825  0.147949  0.265073  0.166746  0.734927  0.166746   
res3a_branch2a  0.264223  0.145573  0.263797  0.187191  0.736203  0.187191   
res3b_branch2a  0.315663  0.173706  0.287301  0.173380  0.712699  0.173380   
res3c_branch2a  0.308010  0.170793  0.291438  0.167031  0.708562  0.167031   
res3d_branch2a  0.344187  0.183094  0.255424  0.174607  0.744576  0.174607   
res4a_branch2a  0.323568  0.143052  0.259878  0.128790  0.740122  0.128790   

                     tpr                 dis            
                    mean       std      mean       std  
res2a_branch2a  0.679943  0.108386  0.478321  0.073505  
res2b_branch2a  0.738919  0.125660  0.448391  0.145494  
res2c_branch2a  0.698175  0.147949  0.438149  0.123504  
res3a_branch2a  0.735777  0.145573  0.419184  0.124017  
res3b_branch2a  0.684337  0.173706  0.468765  0.134106  
res3c_branch2a  0.691990  0.170793  0.462825  0.135529  
res3d_branch2a  0.655813  0.183094  0.477963  0.116945  
res4a_branch2a  0.676432  0.143052  0.442452  0.102850

### Média e desvio padrão nos 59 vídeos

In [29]:
finals = {}
for layer in rf_results.columns.levels[0]:
    finals[layer] = {}
    layer_data = rf_results.loc[:, layer]
    
    metrics_fnr = {}
    metrics_fpr = {}
    metrics_tnr = {}
    metrics_tpr = {} 
    
    pred = layer_data.loc[:, 'predictions']
    true = layer_data.loc[:, 'labels']
    
    for idx,( y_true, y_pred) in enumerate(zip(true, pred)):
        metrics_fnr.update({'{}'.format(idx): fnr(y_true, y_pred)})
        metrics_fpr.update({'{}'.format(idx): fpr(y_true, y_pred)})
        metrics_tpr.update({'{}'.format(idx): tpr(y_true, y_pred)})
        metrics_tnr.update({'{}'.format(idx): tnr(y_true, y_pred)})

        
    finals[layer] = pd.DataFrame({'fnr': metrics_fnr, 'fpr': metrics_fpr, 'tpr': metrics_tpr, 'tnr': metrics_tnr})

z = pd.concat(finals)

In [22]:
dis.agg(lambda x: np.asarray((unumpy.nominal_values(x), unumpy.std_devs(x))))

Exception: Data must be 1-dimensional

In [30]:
rf_control_vid_stats = z.groupby(level=0).agg(['mean', 'std'])

dis = z.loc[:, ['fnr', 'fpr']].agg(lambda x: np.sqrt((x**2).sum()), axis=1).groupby(level=0).agg(['mean', 'std'])
dis.columns = [('dis', name) for name in dis.columns]
# dis = rf_control_vid_stats[['fnr', 'fpr']].groupby(level=0, axis=1).agg(
#     lambda x: [unumpy.uarray(val) for val in x.values]).agg(
#     lambda x: unumpy.sqrt((x**2).sum()), axis=1).agg(
#     lambda x: {'mean': unumpy.nominal_values(x), 'std': unumpy.std_devs(x)})

# dis.columns = [('dis', name) for name in dis.columns]

rf_control_vid_stats = pd.concat([rf_control_vid_stats, dis], axis=1)
rf_control_vid_stats

fnr                 fpr                 tnr            \
                    mean       std      mean       std      mean       std   
res2a_branch2a  0.320485  0.342175  0.325015  0.378387  0.674985  0.378387   
res2b_branch2a  0.269139  0.316303  0.311328  0.388025  0.688672  0.388025   
res2c_branch2a  0.307502  0.330241  0.262781  0.356133  0.737219  0.356133   
res3a_branch2a  0.267158  0.327309  0.262844  0.369681  0.737156  0.369681   
res3b_branch2a  0.320055  0.346342  0.289296  0.381587  0.710704  0.381587   
res3c_branch2a  0.320166  0.343524  0.288266  0.378563  0.711734  0.378563   
res3d_branch2a  0.350436  0.356511  0.255616  0.359698  0.744384  0.359698   
res4a_branch2a  0.329929  0.333124  0.258505  0.347738  0.741495  0.347738   

                     tpr                 dis            
                    mean       std      mean       std  
res2a_branch2a  0.679515  0.342175  0.573471  0.371025  
res2b_branch2a  0.730861  0.316303  0.521036  0.383057  
res2c_branch2a  0.692498  0.330241  0.504681  0.378455  
res3a_branch2a  0.732842  0.327309  0.486121  0.382478  
res3b_branch2a  0.679945  0.346342  0.554944  0.376330  
res3c_branch2a  0.679834  0.343524  0.548956  0.378910  
res3d_branch2a  0.649564  0.356511  0.549944  0.374458  
res4a_branch2a  0.670071  0.333124  0.522294  0.364837

### Média global

In [31]:
rf_control = z.groupby(level=0, axis=0).agg('mean')
rf_control.loc[:, 'dis'] = rf_control.loc[:, ['fnr', 'fpr']].agg(lambda x: np.sqrt((x**2).sum()), axis=1)
rf_control

,fnr,fpr,tnr,tpr,dis
res2a_branch2a,0.320485,0.325015,0.674985,0.679515,0.456449
res2b_branch2a,0.269139,0.311328,0.688672,0.730861,0.411535
res2c_branch2a,0.307502,0.262781,0.737219,0.692498,0.404489
res3a_branch2a,0.267158,0.262844,0.737156,0.732842,0.374781
res3b_branch2a,0.320055,0.289296,0.710704,0.679945,0.431425
res3c_branch2a,0.320166,0.288266,0.711734,0.679834,0.430817
res3d_branch2a,0.350436,0.255616,0.744384,0.649564,0.433757
res4a_branch2a,0.329929,0.258505,0.741495,0.670071,0.419139


## Consistência temporal

### Média e Desvio padrão nos folds

In [32]:
group_sizes = [10, 6, 6, 6, 6, 7, 6, 6, 6]
pos = np.hstack((np.zeros(1), np.cumsum(group_sizes)))

In [33]:
finals = {}
for layer in rf_results.columns.levels[0]:
    finals[layer] = {}
    groups = []
    for idx in range(len(pos)-1):
        layer_data = rf_results.loc[pos[idx]:pos[idx+1]-1, layer]

        metrics_fnr = {}
        metrics_fpr = {}
        metrics_tnr = {}
        metrics_tpr = {}

        pred = layer_data.loc[:, 'predictions'].agg(medfilt, kernel_size=5)
        true = layer_data.loc[:, 'labels']

        for idx,( y_true, y_pred) in enumerate(zip(true, pred)):
            metrics_fnr.update({'{}'.format(idx): fnr(y_true, y_pred)})
            metrics_fpr.update({'{}'.format(idx): fpr(y_true, y_pred)})
            metrics_tpr.update({'{}'.format(idx): tpr(y_true, y_pred)})
            metrics_tnr.update({'{}'.format(idx): tnr(y_true, y_pred)})

        groups += [pd.DataFrame({'fnr': metrics_fnr, 'fpr': metrics_fpr, 'tpr': metrics_tpr, 'tnr': metrics_tnr})]

    finals[layer] = pd.concat(groups, keys=['Black_Backpack', 'Black_Coat', 'Brown_Box',
                                            'Camera_Box', 'Dark-Blue_Box', 'Pink_Bottle',
                                            'Shoe', 'Towel', 'White_Jar'])

z = pd.concat(finals)

In [34]:
rf_temp_group_stats = z.groupby(level=[0,1]).agg('mean').groupby(level=0).agg(['mean', 'std'])
dis = z.groupby(level=[0,1]).agg('mean').loc[:, ['fnr', 'fpr']].agg(
    lambda x: np.sqrt((x**2).sum()), axis=1).groupby(level=0).agg(['mean', 'std'])

dis.columns = [('dis', name) for name in dis.columns]
rf_temp_group_stats = pd.concat([rf_temp_group_stats, dis], axis=1)
rf_temp_group_stats

fnr                 fpr                 tnr            \
                    mean       std      mean       std      mean       std   
res2a_branch2a  0.320270  0.118502  0.316726  0.139712  0.683274  0.139712   
res2b_branch2a  0.259217  0.131745  0.302249  0.219167  0.697751  0.219167   
res2c_branch2a  0.300887  0.157851  0.247769  0.174052  0.752231  0.174052   
res3a_branch2a  0.258747  0.152159  0.245807  0.182145  0.754193  0.182145   
res3b_branch2a  0.318527  0.184995  0.271815  0.179040  0.728185  0.179040   
res3c_branch2a  0.311023  0.181395  0.274483  0.179044  0.725517  0.179044   
res3d_branch2a  0.345234  0.194752  0.249972  0.188304  0.750028  0.188304   
res4a_branch2a  0.323139  0.152025  0.253065  0.147694  0.746935  0.147694   

                     tpr                 dis            
                    mean       std      mean       std  
res2a_branch2a  0.679730  0.118502  0.478232  0.067210  
res2b_branch2a  0.740783  0.131745  0.443273  0.150680  
res2c_branch2a  0.699113  0.157851  0.431618  0.128620  
res3a_branch2a  0.741253  0.152159  0.404437  0.124924  
res3b_branch2a  0.681473  0.184995  0.466012  0.138661  
res3c_branch2a  0.688977  0.181395  0.459606  0.144585  
res3d_branch2a  0.654766  0.194752  0.483060  0.123505  
res4a_branch2a  0.676861  0.152025  0.444364  0.110911

### Média e desvio padrão nos 59 vídeos

In [13]:
finals = {}
for layer in rf_results.columns.levels[0]:
    finals[layer] = {}
    layer_data = rf_results.loc[:, layer]
    
    metrics_fnr = {}
    metrics_fpr = {}
    metrics_tnr = {}
    metrics_tpr = {}

    pred = layer_data.loc[:, 'predictions'].agg(medfilt, kernel_size=5)
    true = layer_data.loc[:, 'labels']
    
    for idx,( y_true, y_pred) in enumerate(zip(true, pred)):
        metrics_fnr.update({'{}'.format(idx): fnr(y_true, y_pred)})
        metrics_fpr.update({'{}'.format(idx): fpr(y_true, y_pred)})
        metrics_tnr.update({'{}'.format(idx): tnr(y_true, y_pred)})
        metrics_tpr.update({'{}'.format(idx): tpr(y_true, y_pred)})
        
    finals[layer] = pd.DataFrame({'fnr': metrics_fnr, 'fpr': metrics_fpr, 'tnr': metrics_tnr, 'tpr': metrics_tpr})

z = pd.concat(finals)

In [14]:
rf_temp_vid_stats = z.groupby(level=0).agg(['mean', 'std'])
dis = z.loc[:, ['fnr', 'fpr']].agg(lambda x: np.sqrt((x**2).sum()), axis=1).groupby(level=0).agg(['mean', 'std'])
dis.columns = [('dis', name) for name in dis.columns]
rf_temp_vid_stats = pd.concat([rf_temp_vid_stats, dis], axis=1)
rf_temp_vid_stats

fnr                 fpr                 tnr            \
                    mean       std      mean       std      mean       std   
res2a_branch2a  0.321046  0.354505  0.318897  0.396206  0.681103  0.396206   
res2b_branch2a  0.267904  0.325642  0.300623  0.400476  0.699377  0.400476   
res2c_branch2a  0.306612  0.345338  0.244923  0.368749  0.755077  0.368749   
res3a_branch2a  0.261024  0.338655  0.245553  0.383779  0.754447  0.383779   
res3b_branch2a  0.322400  0.363004  0.272856  0.389741  0.727144  0.389741   
res3c_branch2a  0.324113  0.359948  0.271762  0.386015  0.728238  0.386015   
res3d_branch2a  0.351744  0.375130  0.250424  0.381233  0.749576  0.381233   
res4a_branch2a  0.329287  0.350403  0.250749  0.367681  0.749251  0.367681   

                     tpr                 dis            
                    mean       std      mean       std  
res2a_branch2a  0.678954  0.354505  0.577846  0.388954  
res2b_branch2a  0.732096  0.325642  0.522109  0.392516  
res2c_branch2a  0.693388  0.345338  0.503179  0.392852  
res3a_branch2a  0.738976  0.338655  0.477355  0.401026  
res3b_branch2a  0.677600  0.363004  0.554457  0.390405  
res3c_branch2a  0.675887  0.359948  0.548238  0.393461  
res3d_branch2a  0.648256  0.375130  0.559519  0.396551  
res4a_branch2a  0.670713  0.350403  0.529598  0.383301

### Média global

In [15]:
rf_temp = z.groupby(level=0, axis=0).agg('mean')
rf_temp.loc[:, 'dis'] = rf_temp.loc[:, ['fnr', 'fpr']].agg(lambda x: np.sqrt((x**2).sum()), axis=1)
rf_temp

,fnr,fpr,tnr,tpr,dis
res2a_branch2a,0.321046,0.318897,0.681103,0.678954,0.452510
res2b_branch2a,0.267904,0.300623,0.699377,0.732096,0.402675
res2c_branch2a,0.306612,0.244923,0.755077,0.693388,0.392426
res3a_branch2a,0.261024,0.245553,0.754447,0.738976,0.358371
res3b_branch2a,0.322400,0.272856,0.727144,0.677600,0.422365
res3c_branch2a,0.324113,0.271762,0.728238,0.675887,0.422970
res3d_branch2a,0.351744,0.250424,0.749576,0.648256,0.431782
res4a_branch2a,0.329287,0.250749,0.749251,0.670713,0.413890


# Resultados da MLP

In [16]:
mlp_results = pd.read_pickle('models/mlp-preds.pkl')

## Grupo controle

Vou primeiramente não fazer nenhum processamento extra e verificar se os valores obtidos batem com resultados já salvos.

### Média e Desvio padrão nos folds

In [17]:
group_sizes = [10, 6, 6, 6, 6, 7, 6, 6, 6]
pos = np.hstack((np.zeros(1), np.cumsum(group_sizes)))

In [18]:
finals = {}
for layer in mlp_results.columns.levels[0]:
    finals[layer] = {}
    groups = []
    for idx in range(len(pos)-1):
        layer_data = mlp_results.loc[pos[idx]:pos[idx+1]-1, layer]

        metrics_fnr = {}
        metrics_fpr = {}
        metrics_tnr = {}
        metrics_tpr = {}

        pred = layer_data.loc[:, 'predictions']
        true = layer_data.loc[:, 'labels']

        for idx,( y_true, y_pred) in enumerate(zip(true, pred)):
            metrics_fnr.update({'{}'.format(idx): fnr(y_true, y_pred)})
            metrics_fpr.update({'{}'.format(idx): fpr(y_true, y_pred)})
            metrics_tpr.update({'{}'.format(idx): tpr(y_true, y_pred)})
            metrics_tnr.update({'{}'.format(idx): tnr(y_true, y_pred)})

        groups += [pd.DataFrame({'fnr': metrics_fnr, 'fpr': metrics_fpr, 'tpr': metrics_tpr, 'tnr': metrics_tnr})]
    finals[layer] = pd.concat(groups, keys=['Black_Backpack', 'Black_Coat', 'Brown_Box',
                                            'Camera_Box', 'Dark-Blue_Box', 'Pink_Bottle',
                                            'Shoe', 'Towel', 'White_Jar'])

z = pd.concat(finals)

In [19]:
mlp_control_group_stats = z.groupby(level=[0,1]).agg('mean').groupby(level=0).agg(['mean', 'std'])
dis = z.groupby(level=[0,1]).agg('mean').loc[:, ['fnr', 'fpr']].agg(
    lambda x: np.sqrt((x**2).sum()), axis=1).groupby(level=0).agg(['mean', 'std'])

dis.columns = [('dis', name) for name in dis.columns]
mlp_control_group_stats = pd.concat([mlp_control_group_stats, dis], axis=1)
mlp_control_group_stats

fnr                 fpr                 tnr            \
                    mean       std      mean       std      mean       std   
res2a_branch2a  0.311300  0.135078  0.367783  0.208127  0.632217  0.208127   
res2b_branch2a  0.290132  0.164712  0.362813  0.188851  0.637187  0.188851   
res2c_branch2a  0.308024  0.216016  0.324715  0.171336  0.675285  0.171336   
res3a_branch2a  0.337004  0.154600  0.290584  0.090367  0.709416  0.090367   
res3b_branch2a  0.244513  0.143626  0.399858  0.168280  0.600142  0.168280   
res3c_branch2a  0.268099  0.139810  0.395127  0.217493  0.604873  0.217493   
res3d_branch2a  0.324606  0.207861  0.365166  0.197326  0.634834  0.197326   
res4a_branch2a  0.309802  0.216434  0.344123  0.223636  0.655877  0.223636   

                     tpr                 dis            
                    mean       std      mean       std  
res2a_branch2a  0.688700  0.135078  0.525104  0.112048  
res2b_branch2a  0.709868  0.164712  0.505655  0.133913  
res2c_branch2a  0.691976  0.216016  0.496127  0.156423  
res3a_branch2a  0.662996  0.154600  0.465746  0.103899  
res3b_branch2a  0.755487  0.143626  0.503022  0.106858  
res3c_branch2a  0.731901  0.139810  0.517883  0.147054  
res3d_branch2a  0.675394  0.207861  0.546736  0.120060  
res4a_branch2a  0.690198  0.216434  0.530938  0.144645

### Média e desvio padrão nos 59 vídeos

In [20]:
finals = {}
for layer in mlp_results.columns.levels[0]:
    finals[layer] = {}
    layer_data = mlp_results.loc[:, layer]
    
    metrics_fnr = {}
    metrics_fpr = {}
    
    pred = layer_data.loc[:, 'predictions']
    true = layer_data.loc[:, 'labels']
    
    for idx,( y_true, y_pred) in enumerate(zip(true, pred)):
        metrics_fnr.update({'{}'.format(idx): fnr(y_true, y_pred)})
        metrics_fpr.update({'{}'.format(idx): fpr(y_true, y_pred)})
        
    finals[layer] = pd.DataFrame({'fnr': metrics_fnr, 'fpr': metrics_fpr})

z = pd.concat(finals)

In [21]:
mlp_control_vid_stats = z.groupby(level=0).agg(['mean', 'std'])
dis = z.loc[:, ['fnr', 'fpr']].agg(lambda x: np.sqrt((x**2).sum()), axis=1).groupby(level=0).agg(['mean', 'std'])
dis.columns = [('dis', name) for name in dis.columns]
mlp_control_vid_stats = pd.concat([mlp_control_vid_stats, dis], axis=1)
mlp_control_vid_stats

fnr                 fpr                 dis          
                    mean       std      mean       std      mean       std
res2a_branch2a  0.297574  0.321404  0.389830  0.420129  0.607200  0.386555
res2b_branch2a  0.298750  0.340662  0.366614  0.405759  0.598412  0.379386
res2c_branch2a  0.295442  0.324118  0.346923  0.384306  0.575732  0.356066
res3a_branch2a  0.342274  0.332097  0.297173  0.342694  0.553692  0.353373
res3b_branch2a  0.244088  0.316517  0.411663  0.415457  0.601697  0.370810
res3c_branch2a  0.270423  0.313693  0.398467  0.402854  0.602786  0.356346
res3d_branch2a  0.342604  0.375467  0.354847  0.384500  0.639473  0.346885
res4a_branch2a  0.298689  0.332874  0.368747  0.368610  0.588509  0.351335

### Média Global

In [22]:
mlp_control = z.groupby(level=0, axis=0).agg('mean')
mlp_control.loc[:, 'dis'] = mlp_control.loc[:, ['fnr', 'fpr']].agg(lambda x: np.sqrt((x**2).sum()), axis=1)
mlp_control

,fnr,fpr,dis
res2a_branch2a,0.297574,0.389830,0.490426
res2b_branch2a,0.298750,0.366614,0.472925
res2c_branch2a,0.295442,0.346923,0.455677
res3a_branch2a,0.342274,0.297173,0.453281
res3b_branch2a,0.244088,0.411663,0.478587
res3c_branch2a,0.270423,0.398467,0.481565
res3d_branch2a,0.342604,0.354847,0.493249
res4a_branch2a,0.298689,0.368747,0.474541


## Consistência temporal

### Média e Desvio padrão nos folds

In [23]:
group_sizes = [10, 6, 6, 6, 6, 7, 6, 6, 6]
pos = np.hstack((np.zeros(1), np.cumsum(group_sizes)))

In [24]:
finals = {}
for layer in mlp_results.columns.levels[0]:
    finals[layer] = {}
    groups = []
    for idx in range(len(pos)-1):
        layer_data = mlp_results.loc[pos[idx]:pos[idx+1]-1, layer]
        
        metrics_fnr = {}
        metrics_fpr = {}
        metrics_tpr = {}
        metrics_tnr = {}
        
        pred = layer_data.loc[:, 'predictions'].agg(medfilt, kernel_size=5)
        true = layer_data.loc[:, 'labels']

        for idx,( y_true, y_pred) in enumerate(zip(true, pred)):
            metrics_fnr.update({'{}'.format(idx): fnr(y_true, y_pred)})
            metrics_fpr.update({'{}'.format(idx): fpr(y_true, y_pred)})
            metrics_tpr.update({'{}'.format(idx): tpr(y_true, y_pred)})
            metrics_tnr.update({'{}'.format(idx): tnr(y_true, y_pred)})
        
        groups += [pd.DataFrame({'fnr': metrics_fnr, 'fpr': metrics_fpr, 'tpr': metrics_tpr, 'tnr': metrics_tnr})]
    finals[layer] = pd.concat(groups, keys=['Black_Backpack', 'Black_Coat', 'Brown_Box',
                                            'Camera_Box', 'Dark-Blue_Box', 'Pink_Bottle',
                                            'Shoe', 'Towel', 'White_Jar'])

z = pd.concat(finals)

In [25]:
mlp_temp_group_stats = z.groupby(level=[0,1]).agg('mean').groupby(level=0).agg(['mean', 'std'])
dis = z.groupby(level=[0,1]).agg('mean').loc[:, ['fnr', 'fpr']].agg(
    lambda x: np.sqrt((x**2).sum()), axis=1).groupby(level=0).agg(['mean', 'std'])

dis.columns = [('dis', name) for name in dis.columns]
mlp_temp_group_stats = pd.concat([mlp_temp_group_stats, dis], axis=1)
mlp_temp_group_stats

fnr                 fpr                 tnr            \
                    mean       std      mean       std      mean       std   
res2a_branch2a  0.310548  0.139147  0.369141  0.214912  0.630859  0.214912   
res2b_branch2a  0.292077  0.168970  0.358830  0.192044  0.641170  0.192044   
res2c_branch2a  0.302626  0.216943  0.316940  0.176936  0.683060  0.176936   
res3a_branch2a  0.338131  0.157877  0.273983  0.095163  0.726017  0.095163   
res3b_branch2a  0.242980  0.149972  0.391906  0.174901  0.608094  0.174901   
res3c_branch2a  0.265239  0.138414  0.382475  0.223281  0.617525  0.223281   
res3d_branch2a  0.326985  0.214130  0.359695  0.212579  0.640305  0.212579   
res4a_branch2a  0.309283  0.223643  0.330490  0.239857  0.669510  0.239857   

                     tpr                 dis            
                    mean       std      mean       std  
res2a_branch2a  0.689452  0.139147  0.527670  0.118756  
res2b_branch2a  0.707923  0.168970  0.505614  0.136590  
res2c_branch2a  0.697374  0.216943  0.488461  0.161215  
res3a_branch2a  0.661869  0.157877  0.457425  0.107994  
res3b_branch2a  0.757020  0.149972  0.500033  0.104896  
res3c_branch2a  0.734761  0.138414  0.507569  0.151335  
res3d_branch2a  0.673015  0.214130  0.550234  0.127577  
res4a_branch2a  0.690717  0.223643  0.527448  0.158304

### Média e desvio padrão nos 59 vídeos

In [26]:
finals = {}
for layer in mlp_results.columns.levels[0]:
    finals[layer] = {}
    layer_data = mlp_results.loc[:, layer]
    
    metrics_fnr = {}
    metrics_fpr = {}

    pred = layer_data.loc[:, 'predictions'].agg(medfilt, kernel_size=5)
    true = layer_data.loc[:, 'labels']
    
    for idx,( y_true, y_pred) in enumerate(zip(true, pred)):
        metrics_fnr.update({'{}'.format(idx): fnr(y_true, y_pred)})
        metrics_fpr.update({'{}'.format(idx): fpr(y_true, y_pred)})
        
    finals[layer] = pd.DataFrame({'fnr': metrics_fnr, 'fpr': metrics_fpr})

z = pd.concat(finals)

In [27]:
mlp_temp_vid_stats = z.groupby(level=0).agg(['mean', 'std'])
dis = z.loc[:, ['fnr', 'fpr']].agg(lambda x: np.sqrt((x**2).sum()), axis=1).groupby(level=0).agg(['mean', 'std'])
dis.columns = [('dis', name) for name in dis.columns]
mlp_temp_vid_stats = pd.concat([mlp_temp_vid_stats, dis], axis=1)
mlp_temp_vid_stats

fnr                 fpr                 dis          
                    mean       std      mean       std      mean       std
res2a_branch2a  0.296726  0.325584  0.391963  0.434358  0.613129  0.397630
res2b_branch2a  0.301115  0.347761  0.362482  0.417989  0.604925  0.386395
res2c_branch2a  0.289503  0.329483  0.340362  0.394368  0.572946  0.365047
res3a_branch2a  0.343352  0.344163  0.281291  0.355703  0.551878  0.368191
res3b_branch2a  0.243390  0.328560  0.404589  0.433374  0.605601  0.386606
res3c_branch2a  0.268701  0.328868  0.387116  0.417558  0.602277  0.373369
res3d_branch2a  0.346202  0.387396  0.348568  0.408615  0.650063  0.364268
res4a_branch2a  0.297138  0.350222  0.357379  0.385920  0.590761  0.369201

### Média Global

In [28]:
mlp_temp = z.groupby(level=0, axis=0).agg('mean')
mlp_temp.loc[:, 'dis'] = mlp_temp.loc[:, ['fnr', 'fpr']].agg(lambda x: np.sqrt((x**2).sum()), axis=1)
mlp_temp

,fnr,fpr,dis
res2a_branch2a,0.296726,0.391963,0.491611
res2b_branch2a,0.301115,0.362482,0.471236
res2c_branch2a,0.289503,0.340362,0.446832
res3a_branch2a,0.343352,0.281291,0.443864
res3b_branch2a,0.243390,0.404589,0.472155
res3c_branch2a,0.268701,0.387116,0.471231
res3d_branch2a,0.346202,0.348568,0.491279
res4a_branch2a,0.297138,0.357379,0.464770


# Tabelas Gerais

### Média e desvio padrão das metricas nos folds

Métricas calculadas para cada fold separadamente e em seguida computada média e variância de cada métrica ao longo dos folds.

In [29]:
group_stats = pd.concat([rf_control_group_stats, rf_temp_group_stats, mlp_control_group_stats, mlp_temp_group_stats],
          keys=['RF s/ consistencia', 'RF c/ consistencia', 'MLP s/ consistencia', 'MLP c/ consistencia'], axis=1)
group_stats

RF s/ consistencia                                          \
                              fnr                 fpr                 tnr   
                             mean       std      mean       std      mean   
res2a_branch2a           0.320057  0.108386  0.321925  0.138613  0.678075   
res2b_branch2a           0.261081  0.125660  0.312946  0.211433  0.687054   
res2c_branch2a           0.301825  0.147949  0.265073  0.166746  0.734927   
res3a_branch2a           0.264223  0.145573  0.263797  0.187191  0.736203   
res3b_branch2a           0.315663  0.173706  0.287301  0.173380  0.712699   
res3c_branch2a           0.308010  0.170793  0.291438  0.167031  0.708562   
res3d_branch2a           0.344187  0.183094  0.255424  0.174607  0.744576   
res4a_branch2a           0.323568  0.143052  0.259878  0.128790  0.740122   

                                                                    ...     \
                               tpr                 dis              ...      
                     std      mean       std      mean       std    ...      
res2a_branch2a  0.138613  0.679943  0.108386  0.478321  0.073505    ...      
res2b_branch2a  0.211433  0.738919  0.125660  0.448391  0.145494    ...      
res2c_branch2a  0.166746  0.698175  0.147949  0.438149  0.123504    ...      
res3a_branch2a  0.187191  0.735777  0.145573  0.419184  0.124017    ...      
res3b_branch2a  0.173380  0.684337  0.173706  0.468765  0.134106    ...      
res3c_branch2a  0.167031  0.691990  0.170793  0.462825  0.135529    ...      
res3d_branch2a  0.174607  0.655813  0.183094  0.477963  0.116945    ...      
res4a_branch2a  0.128790  0.676432  0.143052  0.442452  0.102850    ...      

               MLP c/ consistencia                                          \
                               fnr                 fpr                 tnr   
                              mean       std      mean       std      mean   
res2a_branch2a            0.310548  0.139147  0.369141  0.214912  0.630859   
res2b_branch2a            0.292077  0.168970  0.358830  0.192044  0.641170   
res2c_branch2a            0.302626  0.216943  0.316940  0.176936  0.683060   
res3a_branch2a            0.338131  0.157877  0.273983  0.095163  0.726017   
res3b_branch2a            0.242980  0.149972  0.391906  0.174901  0.608094   
res3c_branch2a            0.265239  0.138414  0.382475  0.223281  0.617525   
res3d_branch2a            0.326985  0.214130  0.359695  0.212579  0.640305   
res4a_branch2a            0.309283  0.223643  0.330490  0.239857  0.669510   

                                                                  
                               tpr                 dis            
                     std      mean       std      mean       std  
res2a_branch2a  0.214912  0.689452  0.139147  0.527670  0.118756  
res2b_branch2a  0.192044  0.707923  0.168970  0.505614  0.136590  
res2c_branch2a  0.176936  0.697374  0.216943  0.488461  0.161215  
res3a_branch2a  0.095163  0.661869  0.157877  0.457425  0.107994  
res3b_branch2a  0.174901  0.757020  0.149972  0.500033  0.104896  
res3c_branch2a  0.223281  0.734761  0.138414  0.507569  0.151335  
res3d_branch2a  0.212579  0.673015  0.214130  0.550234  0.127577  
res4a_branch2a  0.239857  0.690717  0.223643  0.527448  0.158304  

[8 rows x 40 columns]

### Média e desvio padrão da DISTANCIA nos 59 vídeos

Métricas calculadas independentemente em cada um dos 59 vídeos em seguida computada média e variância ao longo de todos os vídeos.

In [30]:
vid_stats = pd.concat([rf_control_vid_stats, rf_temp_vid_stats, mlp_control_vid_stats, mlp_temp_vid_stats],
          keys=['RF s/ consistencia', 'RF c/ consistencia', 'MLP s/ consistencia', 'MLP c/ consistencia'], axis=1)
vid_stats

RF s/ consistencia                                          \
                              fnr                 fpr                 tnr   
                             mean       std      mean       std      mean   
res2a_branch2a           0.320485  0.342175  0.325015  0.378387  0.674985   
res2b_branch2a           0.269139  0.316303  0.311328  0.388025  0.688672   
res2c_branch2a           0.307502  0.330241  0.262781  0.356133  0.737219   
res3a_branch2a           0.267158  0.327309  0.262844  0.369681  0.737156   
res3b_branch2a           0.320055  0.346342  0.289296  0.381587  0.710704   
res3c_branch2a           0.320166  0.343524  0.288266  0.378563  0.711734   
res3d_branch2a           0.350436  0.356511  0.255616  0.359698  0.744384   
res4a_branch2a           0.329929  0.333124  0.258505  0.347738  0.741495   

                                                                    ...     \
                               tpr                 dis              ...      
                     std      mean       std      mean       std    ...      
res2a_branch2a  0.378387  0.679515  0.342175  0.573471  0.371025    ...      
res2b_branch2a  0.388025  0.730861  0.316303  0.521036  0.383057    ...      
res2c_branch2a  0.356133  0.692498  0.330241  0.504681  0.378455    ...      
res3a_branch2a  0.369681  0.732842  0.327309  0.486121  0.382478    ...      
res3b_branch2a  0.381587  0.679945  0.346342  0.554944  0.376330    ...      
res3c_branch2a  0.378563  0.679834  0.343524  0.548956  0.378910    ...      
res3d_branch2a  0.359698  0.649564  0.356511  0.549944  0.374458    ...      
res4a_branch2a  0.347738  0.670071  0.333124  0.522294  0.364837    ...      

               MLP s/ consistencia                                \
                               fpr                 dis             
                              mean       std      mean       std   
res2a_branch2a            0.389830  0.420129  0.607200  0.386555   
res2b_branch2a            0.366614  0.405759  0.598412  0.379386   
res2c_branch2a            0.346923  0.384306  0.575732  0.356066   
res3a_branch2a            0.297173  0.342694  0.553692  0.353373   
res3b_branch2a            0.411663  0.415457  0.601697  0.370810   
res3c_branch2a            0.398467  0.402854  0.602786  0.356346   
res3d_branch2a            0.354847  0.384500  0.639473  0.346885   
res4a_branch2a            0.368747  0.368610  0.588509  0.351335   

               MLP c/ consistencia                                          \
                               fnr                 fpr                 dis   
                              mean       std      mean       std      mean   
res2a_branch2a            0.296726  0.325584  0.391963  0.434358  0.613129   
res2b_branch2a            0.301115  0.347761  0.362482  0.417989  0.604925   
res2c_branch2a            0.289503  0.329483  0.340362  0.394368  0.572946   
res3a_branch2a            0.343352  0.344163  0.281291  0.355703  0.551878   
res3b_branch2a            0.243390  0.328560  0.404589  0.433374  0.605601   
res3c_branch2a            0.268701  0.328868  0.387116  0.417558  0.602277   
res3d_branch2a            0.346202  0.387396  0.348568  0.408615  0.650063   
res4a_branch2a            0.297138  0.350222  0.357379  0.385920  0.590761   

                          
                          
                     std  
res2a_branch2a  0.397630  
res2b_branch2a  0.386395  
res2c_branch2a  0.365047  
res3a_branch2a  0.368191  
res3b_branch2a  0.386606  
res3c_branch2a  0.373369  
res3d_branch2a  0.364268  
res4a_branch2a  0.369201  

[8 rows x 32 columns]

### Métricas Globais

O valor de cada métrica calculada como discutido nas reuniões anteriores:
  * tpr, fpr, fnr, tnr : média ao longo dos 59 vídeos
  * dis : calculada a partir da fnr e fpr médias (calculadas como no item acima)

In [31]:
global_metrics = pd.concat([rf_control, rf_temp, mlp_control, mlp_temp],
          keys=['RF s/ consistencia', 'RF c/ consistencia', 'MLP s/ consistencia', 'MLP c/ consistencia'], axis=1)
global_metrics

RF s/ consistencia                                          \
                              fnr       fpr       tnr       tpr       dis   
res2a_branch2a           0.320485  0.325015  0.674985  0.679515  0.456449   
res2b_branch2a           0.269139  0.311328  0.688672  0.730861  0.411535   
res2c_branch2a           0.307502  0.262781  0.737219  0.692498  0.404489   
res3a_branch2a           0.267158  0.262844  0.737156  0.732842  0.374781   
res3b_branch2a           0.320055  0.289296  0.710704  0.679945  0.431425   
res3c_branch2a           0.320166  0.288266  0.711734  0.679834  0.430817   
res3d_branch2a           0.350436  0.255616  0.744384  0.649564  0.433757   
res4a_branch2a           0.329929  0.258505  0.741495  0.670071  0.419139   

               RF c/ consistencia                                          \
                              fnr       fpr       tnr       tpr       dis   
res2a_branch2a           0.321046  0.318897  0.681103  0.678954  0.452510   
res2b_branch2a           0.267904  0.300623  0.699377  0.732096  0.402675   
res2c_branch2a           0.306612  0.244923  0.755077  0.693388  0.392426   
res3a_branch2a           0.261024  0.245553  0.754447  0.738976  0.358371   
res3b_branch2a           0.322400  0.272856  0.727144  0.677600  0.422365   
res3c_branch2a           0.324113  0.271762  0.728238  0.675887  0.422970   
res3d_branch2a           0.351744  0.250424  0.749576  0.648256  0.431782   
res4a_branch2a           0.329287  0.250749  0.749251  0.670713  0.413890   

               MLP s/ consistencia                     MLP c/ consistencia  \
                               fnr       fpr       dis                 fnr   
res2a_branch2a            0.297574  0.389830  0.490426            0.296726   
res2b_branch2a            0.298750  0.366614  0.472925            0.301115   
res2c_branch2a            0.295442  0.346923  0.455677            0.289503   
res3a_branch2a            0.342274  0.297173  0.453281            0.343352   
res3b_branch2a            0.244088  0.411663  0.478587            0.243390   
res3c_branch2a            0.270423  0.398467  0.481565            0.268701   
res3d_branch2a            0.342604  0.354847  0.493249            0.346202   
res4a_branch2a            0.298689  0.368747  0.474541            0.297138   

                                    
                     fpr       dis  
res2a_branch2a  0.391963  0.491611  
res2b_branch2a  0.362482  0.471236  
res2c_branch2a  0.340362  0.446832  
res3a_branch2a  0.281291  0.443864  
res3b_branch2a  0.404589  0.472155  
res3c_branch2a  0.387116  0.471231  
res3d_branch2a  0.348568  0.491279  
res4a_branch2a  0.357379  0.464770

# Limiares de Decisão

In [32]:
thres_rf = {
    'res2a_branch2a': np.array([ 0.47,  0.48,  0.51,  0.45,  0.45,  0.47,  0.45,  0.5 ,  0.48]),
    'res2b_branch2a': np.array([ 0.46,  0.47,  0.47,  0.45,  0.44,  0.47,  0.45,  0.45,  0.47]),
    'res2c_branch2a': np.array([ 0.5 ,  0.5 ,  0.52,  0.46,  0.5 ,  0.46,  0.46,  0.45,  0.47]),
    'res3a_branch2a': np.array([ 0.44,  0.47,  0.48,  0.47,  0.46,  0.46,  0.46,  0.43,  0.48]),
    'res3b_branch2a': np.array([ 0.43,  0.47,  0.43,  0.46,  0.45,  0.45,  0.45,  0.39,  0.47]),
    'res3c_branch2a': np.array([ 0.47,  0.43,  0.44,  0.46,  0.42,  0.49,  0.43,  0.41,  0.41]),
    'res3d_branch2a': np.array([ 0.43,  0.5 ,  0.44,  0.46,  0.44,  0.49,  0.45,  0.44,  0.46]),
    'res4a_branch2a': np.array([ 0.43,  0.44,  0.46,  0.46,  0.43,  0.46,  0.45,  0.46,  0.41])
    }

In [33]:
thres_mlp = {
    'res2a_branch2a': np.array([ 0.18814139,  0.31943369,  0.39504543,  0.35135305,  0.33910069,
         0.17238259,  0.22892506,  0.15456134,  0.18548249]),
    'res2b_branch2a': np.array([ 0.25836685,  0.3727459 ,  0.38897648,  0.22101182,  0.17316011,
         0.19597945,  0.22360186,  0.18484314,  0.20561291]),
    'res2c_branch2a': np.array([ 0.21400851,  0.16561291,  0.37908047,  0.31742853,  0.20963322,
         0.13999635,  0.30783173,  0.16720267,  0.19069536]),
    'res3a_branch2a': np.array([ 0.21648423,  0.21563871,  0.29692313,  0.26062375,  0.18006811,
         0.16455787,  0.23955758,  0.18813033,  0.17120564]),
    'res3b_branch2a': np.array([ 0.23373182,  0.19150411,  0.3928853 ,  0.20004682,  0.34221199,
         0.21564305,  0.20647231,  0.15977399,  0.25714439]),
    'res3c_branch2a': np.array([ 0.22059107,  0.21895106,  0.51026702,  0.16321161,  0.28893679,
         0.1923275 ,  0.23382066,  0.1363868 ,  0.21580109]),
    'res3d_branch2a': np.array([ 0.26357952,  0.20225812,  0.32853153,  0.2399817 ,  0.21420002,
         0.1387859 ,  0.19585033,  0.17571434,  0.29557118]),
    'res4a_branch2a': np.array([ 0.18346438,  0.25938639,  0.34845608,  0.24746382,  0.26145741,
         0.10502549,  0.20290631,  0.12660566,  0.22551709])
    }

Tabela explicitando o limiar em cada fold para cada camada

In [34]:
thres = pd.concat([pd.DataFrame(thres_rf), pd.DataFrame(thres_mlp)], keys=['RF', 'MLP'], axis=0)
thres

res2a_branch2a  res2b_branch2a  res2c_branch2a  res3a_branch2a  \
RF  0        0.470000        0.460000        0.500000        0.440000   
    1        0.480000        0.470000        0.500000        0.470000   
    2        0.510000        0.470000        0.520000        0.480000   
    3        0.450000        0.450000        0.460000        0.470000   
    4        0.450000        0.440000        0.500000        0.460000   
    5        0.470000        0.470000        0.460000        0.460000   
    6        0.450000        0.450000        0.460000        0.460000   
    7        0.500000        0.450000        0.450000        0.430000   
    8        0.480000        0.470000        0.470000        0.480000   
MLP 0        0.188141        0.258367        0.214009        0.216484   
    1        0.319434        0.372746        0.165613        0.215639   
    2        0.395045        0.388976        0.379080        0.296923   
    3        0.351353        0.221012        0.317429        0.260624   
    4        0.339101        0.173160        0.209633        0.180068   
    5        0.172383        0.195979        0.139996        0.164558   
    6        0.228925        0.223602        0.307832        0.239558   
    7        0.154561        0.184843        0.167203        0.188130   
    8        0.185482        0.205613        0.190695        0.171206   

       res3b_branch2a  res3c_branch2a  res3d_branch2a  res4a_branch2a  
RF  0        0.430000        0.470000        0.430000        0.430000  
    1        0.470000        0.430000        0.500000        0.440000  
    2        0.430000        0.440000        0.440000        0.460000  
    3        0.460000        0.460000        0.460000        0.460000  
    4        0.450000        0.420000        0.440000        0.430000  
    5        0.450000        0.490000        0.490000        0.460000  
    6        0.450000        0.430000        0.450000        0.450000  
    7        0.390000        0.410000        0.440000        0.460000  
    8        0.470000        0.410000        0.460000        0.410000  
MLP 0        0.233732        0.220591        0.263580        0.183464  
    1        0.191504        0.218951        0.202258        0.259386  
    2        0.392885        0.510267        0.328532        0.348456  
    3        0.200047        0.163212        0.239982        0.247464  
    4        0.342212        0.288937        0.214200        0.261457  
    5        0.215643        0.192328        0.138786        0.105025  
    6        0.206472        0.233821        0.195850        0.202906  
    7        0.159774        0.136387        0.175714        0.126606  
    8        0.257144        0.215801        0.295571        0.225517

In [35]:
media = thres.groupby(level=0).agg(np.mean, axis=0)
desvio = thres.groupby(level=0).agg(np.std, axis=0)
thres_stats = pd.concat([media, desvio], keys=['mean', 'std']).swaplevel(0,1).sort_index().T
thres_stats

MLP                  RF          
                    mean       std      mean       std
res2a_branch2a  0.259381  0.086181  0.473333  0.020548
res2b_branch2a  0.247144  0.075243  0.458889  0.010999
res2c_branch2a  0.232388  0.077661  0.480000  0.023570
res3a_branch2a  0.214799  0.041903  0.461111  0.015947
res3b_branch2a  0.244379  0.071557  0.444444  0.023623
res3c_branch2a  0.242255  0.103141  0.440000  0.026247
res3d_branch2a  0.228275  0.056432  0.456667  0.022608
res4a_branch2a  0.217809  0.069985  0.444444  0.017069

## Salvando tudo


In [36]:
thres_stats.to_csv('thres_stats.csv')
global_metrics.to_csv('global_metrics.csv')
vid_stats.to_csv('vid_stats.csv')
group_stats.to_csv('group_stats.csv')